In [ ]:

import numpy as np
import matplotlib.pyplot as plt

def pairCorrelationFunction_2D(x, y, S, rMax, dr):
    """Compute the two-dimensional pair correlation function, also known
    as the radial distribution function, for a set of circular particles
    contained in a square region of a plane.  This simple function finds
    reference particles such that a circle of radius rMax drawn around the
    particle will fit entirely within the square, eliminating the need to
    compensate for edge effects.  If no such particles exist, an error is
    returned. Try a smaller rMax...or write some code to handle edge effects! ;)

    Arguments:
        x               an array of x positions of centers of particles
        y               an array of y positions of centers of particles
        S               length of each side of the square region of the plane
        rMax            outer diameter of largest annulus
        dr              increment for increasing radius of annulus

    Returns a tuple: (g, radii, interior_indices)
        g(r)            a numpy array containing the correlation function g(r)
        radii           a numpy array containing the radii of the
                        annuli used to compute g(r)
        reference_indices   indices of reference particles
    """
    from numpy import zeros, sqrt, where, pi, mean, arange, histogram
    # Number of particles in ring/area of ring/number of reference particles/number density
    # area of ring = pi*(r_outer**2 - r_inner**2)

    # Find particles which are close enough to the box center that a circle of radius
    # rMax will not cross any edge of the box
    bools1 = x > rMax
    bools2 = x < (S - rMax)
    bools3 = y > rMax
    bools4 = y < (S - rMax)
    interior_indices, = where(bools1 * bools2 * bools3 * bools4)
    num_interior_particles = len(interior_indices)

    if num_interior_particles < 1:
        raise  RuntimeError ("No particles found for which a circle of radius rMax\
                will lie entirely within a square of side length S.  Decrease rMax\
                or increase the size of the square.")

    edges = arange(0., rMax + 1.1 * dr, dr)
    num_increments = len(edges) - 1
    g = zeros([num_interior_particles, num_increments])
    radii = zeros(num_increments)
    numberDensity = len(x) / S**2

    # Compute pairwise correlation for each interior particle
    for p in range(num_interior_particles):
        index = interior_indices[p]
        d = sqrt((x[index] - x)**2 + (y[index] - y)**2)
        d[index] = 2 * rMax

        (result, bins) = histogram(d, bins=edges, normed=False)
        g[p, :] = result/numberDensity

    # Average g(r) for all interior particles and compute radii
    g_average = zeros(num_increments)
    for i in range(num_increments):
        radii[i] = (edges[i] + edges[i+1]) / 2.
        rOuter = edges[i + 1]
        rInner = edges[i]
        g_average[i] = mean(g[:, i]) / (pi * (rOuter**2 - rInner**2))

    return (g_average, radii, interior_indices)
####

def pairCorrelationFunction_3D(x, y, z, S, rMax, dr):
    """Compute the three-dimensional pair correlation function for a set of
    spherical particles contained in a cube with side length S.  This simple
    function finds reference particles such that a sphere of radius rMax drawn
    around the particle will fit entirely within the cube, eliminating the need
    to compensate for edge effects.  If no such particles exist, an error is
    returned.  Try a smaller rMax...or write some code to handle edge effects! ;)

    Arguments:
        x               an array of x positions of centers of particles
        y               an array of y positions of centers of particles
        z               an array of z positions of centers of particles
        S               length of each side of the cube in space
        rMax            outer diameter of largest spherical shell
        dr              increment for increasing radius of spherical shell

    Returns a tuple: (g, radii, interior_indices)
        g(r)            a numpy array containing the correlation function g(r)
        radii           a numpy array containing the radii of the
                        spherical shells used to compute g(r)
        reference_indices   indices of reference particles
    """
    from numpy import zeros, sqrt, where, pi, mean, arange, histogram

    # Find particles which are close enough to the cube center that a sphere of radius
    # rMax will not cross any face of the cube
    bools1 = x > rMax
    bools2 = x < (S - rMax)
    bools3 = y > rMax
    bools4 = y < (S - rMax)
    bools5 = z > rMax
    bools6 = z < (S - rMax)

    interior_indices, = where(bools1 * bools2 * bools3 * bools4 * bools5 * bools6)
    num_interior_particles = len(interior_indices)

    if num_interior_particles < 1:
        raise  RuntimeError ("No particles found for which a sphere of radius rMax\
                will lie entirely within a cube of side length S.  Decrease rMax\
                or increase the size of the cube.")

    edges = arange(0., rMax + 1.1 * dr, dr)
    num_increments = len(edges) - 1
    g = zeros([num_interior_particles, num_increments])
    radii = zeros(num_increments)
    numberDensity = len(x) / S**3

    # Compute pairwise correlation for each interior particle
    for p in range(num_interior_particles):
        index = interior_indices[p]
        d = sqrt((x[index] - x)**2 + (y[index] - y)**2 + (z[index] - z)**2)
        d[index] = 2 * rMax

        (result, bins) = histogram(d, bins=edges, normed=False)
        g[p,:] = result / numberDensity

    # Average g(r) for all interior particles and compute radii
    g_average = zeros(num_increments)
    for i in range(num_increments):
        radii[i] = (edges[i] + edges[i+1]) / 2.
        rOuter = edges[i + 1]
        rInner = edges[i]
        g_average[i] = mean(g[:, i]) / (4.0 / 3.0 * pi * (rOuter**3 - rInner**3))

    return (g_average, radii, interior_indices)
    # Number of particles in shell/total number of particles/volume of shell/number density
    # shell volume = 4/3*pi(r_outer**3-r_inner**3)
#### 
from numpy import *
def generate_hex_circle_packing(a, width):
    """Generate a domain of a given width filled with hexagonally packed
    circles.  The height will be determined so that the vertical 
    boundary condition is periodic.

    Arguments:
    a       particle radius
    width   domain width, in terms of particle radius

    Returns:
    x_list  list of x coordinates
    y_list  list of y coordinates
    x_size  width of domain (equal to argument width)
    y_size  height of domain
    """
    numParticles = 0

    x_list = []
    y_list = []
    y = a
    x = a
    rowNumber = 0
    # Create a row
    while y <= width*1.01:
        # Create circles in a row
        while x < width:
            x_list.append(x)
            x = x + 2*a
            y_list.append(y)
            numParticles = numParticles + 1
        y = y + a*np.sqrt(3.)
        rowNumber = rowNumber + 1
        if rowNumber%2 == 0:
            x = a
        else:
            x = 0
    x_size = width
    y_size = rowNumber*a*np.sqrt(3)

    return array(x_list), array(y_list), x_size, y_size

def plot_adsorbed_circles(adsorbed_x, adsorbed_y, radius, width, reference_indices=[]):
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.patches import Circle

    # Plot each run
    fig = plt.figure()
    ax = fig.add_subplot(111)
    for p in range(len(adsorbed_x)):
        if len(np.where(reference_indices == p)[0]) > 0:
            ax.add_patch(Circle((adsorbed_x[p], adsorbed_y[p]), radius, 
                edgecolor='black', facecolor='black'))
        else:
            ax.add_patch(Circle((adsorbed_x[p], adsorbed_y[p]), radius,
                edgecolor='black', facecolor='white'))

    ax.set_aspect(1.0)
    plt.axhline(y=0, color='k')
    plt.axhline(y=width, color='k')
    plt.axvline(x=0, color='k')
    plt.axvline(x=width, color='k')
    plt.axis([-0.1*width, width*1.1, -0.1*width, width*1.1])
    plt.xlabel("non-dimensional x")
    plt.ylabel("non-dimensional y")

    return ax


# Particle setup
domain_size = 20.0
num_particles = 2000

# Calculation setup
dr = 0.1

### Random arrangement of particles ###
particle_radius = 0.1
rMax = domain_size / 4
x = np.random.uniform(low=0, high=domain_size, size=num_particles)
y = np.random.uniform(low=0, high=domain_size, size=num_particles)

# Compute pair correlation
g_r, r, reference_indices = pairCorrelationFunction_2D(x, y, domain_size, rMax, dr)

# Visualize
plt.figure()
plt.plot(r, g_r, color='black')
plt.xlabel('r')
plt.ylabel('g(r)')
plt.xlim( (0, rMax) )
plt.ylim( (0, 1.05 * g_r.max()) )

plot_adsorbed_circles(x, y, particle_radius, domain_size, reference_indices=reference_indices)

### Hexagonal circle packing ###
particle_radius = 1.0
domain_size = 50.0
rMax = domain_size / 3

x, y, domain_width, domain_height = generate_hex_circle_packing(particle_radius, domain_size)

# Compute pair correlation
g_r, r, reference_indices = pairCorrelationFunction_2D(x, y, domain_size,
        rMax, dr)

# Visualize
plt.figure()
plt.plot(r, g_r, color='black')
plt.xlabel('r')
plt.ylabel('g(r)')
plt.xlim( (0, rMax) )
plt.ylim( (0, 1.05 * g_r.max()) )

plot_adsorbed_circles(x, y, particle_radius, domain_size, reference_indices=reference_indices)

plt.show()

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
# Particle setup
domain_size = 20.0
num_particles = 2000

# Calculation setup
dr = 0.1

### Random arrangement of particles ###
particle_radius = 0.1
rMax = domain_size / 4
x = np.random.uniform(low=0, high=domain_size, size=num_particles)
y = np.random.uniform(low=0, high=domain_size, size=num_particles)

# Compute pair correlation
g_r, r, reference_indices = pairCorrelationFunction_2D(x, y, domain_size, rMax, dr)

# Visualize
plt.figure()
plt.plot(r, g_r, color='black')
plt.xlabel('r')
plt.ylabel('g(r)')
plt.xlim( (0, rMax) )
plt.ylim( (0, 1.05 * g_r.max()) )